# Demo 02 - Adaptive Spectrum Processing

This notebook presents an example to read in a sweep of raw data and perform the Adaptive Spectrum Processing method, which was published by Kong++12. The original algorithm description is documented in the paper:

\[Kong++12\] Kong, F., Y. Zhang, R. Palmer, "Wind Turbine Clutter Mitigation for Weather Radar by Adaptive Spectrum Processing," __2012 IEEE Radar Conference__, Atlanta, GA, 2012.

The process can be summarized as follows:

 - Contaminated cells are hand-picked where it has been indicated to contain wind turbines.
 - The clean area is derived from expanding the contaminated area using dilation.
 - Average velocity distribution is derived using fft(), incoherently averaged.
 - Newly added for practicality: a further smoothing is accomplished using a rolling average, m-tap.

In [ ]:
import os
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi
import cspec

zmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.zmap()[:, :3])
vmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.vmap()[:, :3])

# file = '/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/07/20200616_071735.439598-9E-02.iqData.XXXX.AKITA.dat'
file = os.path.expanduser('~/Downloads/20200616_071735.439598-9E-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

In [ ]:
s = time.time()
ray_pulses, cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

# Go through the pulses for azimuth
a = np.zeros(len(ray_pulses), dtype=np.single)
for k, pulses in enumerate(ray_pulses):
    a[k] = pulses[0].azimuth
    
# Choose ray 2 to whatever that completes the 360-deg coverage
n = np.argmin(np.abs(a[3:] - a[2])) + 1
a = a[2:n+2]
ray_pulses = ray_pulses[2:n+2]
cpi_headers = cpi_headers[2:n+2]

In [ ]:
# Dimensions
naz = len(ray_pulses)
ngate_long = ray_pulses[0][0].ngate_long_hi
ngate_short = ray_pulses[0][0].ngate_short_hi
ngate = ngate_long + ngate_short

# Elevation assumed to be flat from the very first pulse
scan_el = ray_pulses[0][0].elevation
scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

# Sampling code from the CPI header
fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
dr = 3.0e8 / fs / 2
r = 1.0e-3 * np.arange(0, ngate, dtype=np.single) * dr

In [ ]:
# Noise estimate, try azimuth 0, around 20-25 km
ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

# Gather the samples. Ignore phase code since we are only interested in amplitude
for k, pulse in enumerate(ray_pulses[ia]):
    samples[k, :] = pulse.h_long_hi[ir:er]
noise = np.mean(np.abs(samples)) ** 2
print('Noise estimate in 16-bit ADU: {:.4f}'.format(noise))

In [ ]:
s = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

# Go through the pulses
a = np.zeros(len(ray_pulses), dtype=np.single)
for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    a[k] = pulses[0].azimuth
    for j, pulse in enumerate(pulses):
        p[j, ngate_short:] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
        p[j, :ngate_short] = pulse.h_short_hi * np.exp(-1j * pulse.phase_h_short)
    # pp = pp - np.mean(pp, axis=0)
    pp = p[1:, :] * np.conj(p[:-1, :])
    s[k, :] = np.mean(np.abs(p) ** 2, axis=0)
    v[k, :] = np.angle(np.sum(pp, axis=0))

# Signal
s -= noise
s[s <= 0] = 1.0e-6                                    # Avoid log(0)
snr = 10 * np.log10(s / noise)                        # Signal-to-noise ratio in dB
z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
z[:, :ngate_short] += 30                              # 30-dB on the short waveform, perhaps?

# Thresholding at SNR = 0 dB
m = snr < 0
z[m] = np.nan
v[m] = np.nan

In [ ]:
# Radar cell locations
ce = np.cos(np.deg2rad(scan_el))
rr, aa = np.meshgrid(r, np.deg2rad(a))
xx = rr * ce * np.sin(aa)
yy = rr * ce * np.cos(aa)

# Turbine locations
turbines = toshi.turbine_cells
a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

# Various domain to choose from
xlim, ylim = (-13, 5), (3, 18)
# xlim, ylim = (-25, 10), (0, 35)
# xlim, ylim = (-20, 20), (-20, 20)

# Reflectivity plot
plt.figure(figsize=(5.2, 4), dpi=200)
plt.pcolormesh(xx[:, :500], yy[:, :500], z[:, :500], cmap=zmap)
plt.plot(x_turb, y_turb, '.k', markersize=4)
plt.clim((-32, 96))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

# Velocity plot
plt.figure(figsize=(5.2, 4), dpi=200)
plt.pcolormesh(xx[::2, :500], yy[::2, :500], v[::2, :500], cmap=vmap)
plt.plot(x_turb, y_turb, '.k', markersize=4)
plt.clim((-5, 5))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

plt.show()

In [ ]:
cells = cspec.pos2cellid(turbines, a, r)

m = np.zeros(z.shape, dtype=bool)
for c in cells:
    m[c[0], c[1]] = True

# Negate all cells before 12-km
m[:, :160] = False

m[:, 1:] = np.logical_or(m[:, 1:], m[:, :-1])
m[:, 1:] = np.logical_or(m[:, 1:], m[:, :-1])
m[:, 1:] = np.logical_or(m[:, 1:], m[:, :-1])
m[:, 1:] = np.logical_or(m[:, 1:], m[:, :-1])
m = cspec.dilate(m)
m = cspec.dilate(m)
m = cspec.dilate(m)
m[:, 1:] = np.logical_or(m[:, 1:], m[:, :-1])

m1 = cspec.dilate(m)
m2 = cspec.dilate(m1)

tags = cspec.mask2tags(m1)

# Take out the dilated portion
tags[m1 ^ m] = 0

In [ ]:
# Gather the 1st cluster
mask = tags == 1
dmask = cspec.dilate(cspec.dilate(mask))

gmask = dmask ^ mask

# Overide the beginning and end of the cluster since there are so much ground clutter
gmask[:, :160] = False
gmask[:, 240:] = False
gmask[120:, :] = False
gmask[::2, :] = False

# Cluster tags in 5-interval
img = 5.0 * (mask + 2.0 * gmask)
img[img < 1] = np.nan

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, img, cmap=zmap)
plt.clim((-32, 96))
# plt.pcolormesh(xx[::2, :500], yy[::2, :500], -5 * img[::2, :500] + v[::2, :500], cmap=vmap)
# plt.clim((-5, 5))
plt.xlim((-13, 5))
plt.ylim((3, 18))
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title('Contaminated and Clean Cells', fontsize=8)

plt.show()

In [ ]:
# tt = np.unique(tags)

In [ ]:
dirty_cells = cspec.mask2cells(mask)
clean_cells = cspec.mask2cells(gmask)

In [ ]:
# Windowed pulses
w = scipy.signal.get_window('hamming', len(p)).reshape((p.shape[0],))
# wp = p * w

In [ ]:
nfft = 256
filts = []
gs = [10, 20, 40, 80]

for g in gs:
    spec = np.zeros(nfft, dtype=np.single)
    for (ia, ir) in clean_cells[:g]:
        p = np.zeros(len(ray_pulses[ia]), dtype=np.csingle)
        for k, pulse in enumerate(ray_pulses[ia]):
            p[k] = pulse.h_long_hi[ir - ngate_short] * np.exp(-1j * pulse.phase_h_long)
        p -= np.mean(p)
        w = scipy.signal.get_window('blackmanharris', len(p)).reshape((p.shape[0],))
        spec += np.abs(np.fft.fft(p * w, nfft))
    
    # m-tap circular averaging
    m = 5
    filt = spec.copy()
    for k in range(1, m):
        filt += np.roll(spec, k)
    filt = np.roll(filt, -int(m/2))    # Compensate for running-average lag
    filt /= m
    filt /= np.sqrt(np.mean(filt ** 2))   # Normalize by noise gain
    # filt /= np.sqrt(np.max(filt ** 2))    # Normalize by peak power
    
    filts.append(filt)

In [ ]:
omega = np.arange(-nfft/2, nfft/2) / nfft * np.pi

plt.figure(figsize=(8, 3), dpi=100)
for k, g in enumerate(gs):
    plt.plot(omega, 20.0 * np.log10(np.fft.fftshift(filts[k])), label=g)
plt.xlim((-1.6, 1.6))
plt.ylim((-45, 16))
plt.grid(linewidth=0.25)
plt.xlabel('Omega (rad/sample)')
plt.ylabel('Filter Amplitude (dB)')
plt.title('Adaptive Filter Derived from the Clean Cells (m = {})'.format(m))
plt.legend(gs)
plt.show()

In [ ]:
sc = s.copy() + noise
use_window = True

for (ia, ir) in dirty_cells:
    p = np.zeros(len(ray_pulses[ia]), dtype=np.csingle)
    for k, pulse in enumerate(ray_pulses[ia]):
        p[k] = pulse.h_long_hi[ir - ngate_short] * np.exp(-1j * pulse.phase_h_long)
    # For sanity check, this should be identical to the original method
    if use_window:
        w = scipy.signal.get_window('blackmanharris', len(p)).reshape((p.shape[0],))
        w /= np.sqrt(np.mean(w ** 2))
        spec = np.abs(np.fft.fft(p * w, nfft))
    else:
        spec = np.abs(np.fft.fft(p, nfft))
    # Filtering in spectral domain
    spec *= filt
    sc[ia, ir] = np.sum(spec ** 2 / len(p) / nfft)

# Signal
sc -= noise
sc[sc <= 0] = 1.0e-6                                    # Avoid log(0)

# Reflectivity calculation
snr = 10 * np.log10(sc / noise)                         # Signal-to-noise ratio
zc = 10 * np.log10(sc * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
zc[:, :ngate_short] += 30                               # 30-dB on the short waveform, perhaps?

# Thresholding
m = snr < 0
zc[m] = np.nan

In [ ]:
plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx[:, :500], yy[:, :500], zc[:, :500], cmap=zmap)
# plt.plot(x_turb, y_turb, '.k')
plt.clim((-32, 96))
plt.xlim((-13, 5))
plt.ylim((3, 18))
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)
plt.show()

In [ ]:
plt.close()